# DarkNet Model definition

In [ ]:
# Mount the Google Drive directory '/content/drive' to Google Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#This line of code imports the fastai library 
from fastai.vision import *

In [ ]:
#Stablishes the directory of the training and test images on the path variable
path = Path('/content/drive/MyDrive/img')

In [ ]:
#Sets the random seed of the NumPy pseudo-random number generator so NumPy 
#generate pseudo-random numbers for random processes.
np.random.seed(41)
#Grabs all the images in the 'path' directory (train and test images) 
#and normalizes them to 256x256 pixels where bs is the batch size and 
#num_workersn describes how many subprocesses to use for data loading.
data = ImageDataBunch.from_folder(path, train="train", valid ="test",
        ds_tfms=get_transforms(), size=(256,256), bs=32, num_workers=4).normalize()

/usr/local/lib/python3.7/dist-packages/fastai/vision/transform.py:247: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:760.)
  return _solve_func(B,A)[0][:,0]
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3658: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for deta

In [ ]:
#DarkCovidNet (modified Darknet model)
#Model obtained from https://discuss.pytorch.org/t/load-a-single-image-in-a-pretrained-pytorch-net-always-get-0-of-predict/92740
def conv_block(ni, nf, size=3, stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return nn.Sequential(
        nn.Conv2d(ni, nf, kernel_size=size, stride=stride,
                  padding=(for_pad(size) - 1)//2, bias=False), 
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(negative_slope=0.1, inplace=True)  
    )
    
def triple_conv(ni, nf):
    return nn.Sequential(
        conv_block(ni, nf),
        conv_block(nf, ni, size=1),  
        conv_block(ni, nf)
    )

def maxpooling():
    return nn.MaxPool2d(2, stride=2)

model = nn.Sequential(
    conv_block(3, 8),
    maxpooling(),
    conv_block(8, 16),
    maxpooling(),
    triple_conv(16, 32),
    maxpooling(),
    triple_conv(32, 64),
    maxpooling(),
    triple_conv(64, 128),
    maxpooling(),
    triple_conv(128, 256),
    conv_block(256, 128, size=1),
    conv_block(128, 256),
    conv_layer(256, 5),
    Flatten(),
    nn.Linear(845, 5)
)

In [ ]:
#Put the CNN model into a Learner along with the data, loss function and metrics
learn = Learner(data, model, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)

# Model Loading

In [ ]:
#Loads the previosly saved model
learn=learn.load('/content/drive/MyDrive/img/DarkNet')

# Anvil Connection

In [ ]:
#Connects with the Anvil form 
from getpass import getpass
uplink_key = getpass('Enter your Uplink key: ')
#FI5VVWPMVBKFU5PL5BL3325N-266GD4RAMZWARNXS

Enter your Uplink key: ··········


In [ ]:
#Installation of Anvil UpLink
!pip install anvil-uplink

     |████████████████████████████████| 62 kB 1.0 MB/s 
     |████████████████████████████████| 51 kB 234 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45230 sha256=a78bbd5679669af4614a5ab12003b319aab4b239561e03eda789f0e2d891e1b3
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


In [ ]:
#Imports the Anvil Server and connects to the Anvil Form
import anvil.server
anvil.server.connect("FI5VVWPMVBKFU5PL5BL3325N-266GD4RAMZWARNXS")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (published)" as SERVER


In [ ]:
#Test Single image
import numpy as np
import cv2 as cv
from fastai.vision import *
import PIL.Image
import anvil.media
from skimage import transform

In [ ]:
#Allows calling the function predict_neumonia from the web
@anvil.server.callable
#Creates a callable function predict_neumonia that imports a file
def predict_neumonia(file):
  #Writes a temporary file as f
  with anvil.media.TempFile(file) as f:
    #Uses the function cv.imread from cv2 to read the imported file and saves it in img_cv2
    img_cv2 = cv.imread(f)
    #Convert PIL style image array to torch style image tensor.
    img_fastai = Image(pil2tensor(img_cv2, dtype=np.float32).div_(255))

    #The imported image is processed by the trained DarkNet
    r=learn.predict(img_fastai)

    #Class labels
    labels={'Random':0,
            'COVID':1,
            'Normal':2,
            'Bacterial Pneumonia':3,
            'Viral Pneumonia':4
            }
    names= dict((v,k) for k,v in labels.items())

    #Obtains the belonging percentage of the image to each class
    scores=r[2]
    scores=scores.numpy()

    #Obtains the index of the class with the greater belonging percentage
    mayor=0;
    index1=0;
    for i in range(5):

    	if (round(scores[i]*100,2)>mayor):
	    	mayor=round(scores[i]*100,2);
	    	index1=i;


    #Returns the belongins percentages and the index of the class with the 
    #greater belonging percentage
    return names[1] +": "+ str(round(scores[1]*100,2))+"%"+"\n"+names[2] +": "+ str(round(scores[2]*100,2))+"%"+"\n"+names[3] +": "+ str(round(scores[3]*100,2))+"%"+"\n"+names[4] +": "+ str(round(scores[4]*100,2))+"%", index1

In [ ]:
#A shortcut to keep the Python script running, to allow the app call functions in it.
anvil.server.wait_forever()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3658: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
